In [1]:
PROJECT=!gcloud config get-value core/project 
#gcloud config list project --format "value(core.project)"
print("Your current GCP Project Name is: ", PROJECT)

Your current GCP Project Name is:  ['fml-gcp-project-01']


In [2]:
!gsutil mb gs://pedok-ml-gcp-vision-api
!gsutil ls

Creating gs://pedok-ml-gcp-vision-api/...


gs://pedok-ml-gcp-vision-api/


In [3]:
!gcloud services enable vision.googleapis.com

In [4]:
#!gcloud iam service-accounts create my-vision-sa --display-name "my vision service account"

In [5]:
!gcloud iam service-accounts keys create key.json --iam-account my-vision-sa@fml-gcp-project-01.iam.gserviceaccount.com
#!gcloud iam service-accounts keys create ~/key.json --iam-account my-vision-sa@{PROJECT_ID}.iam.gserviceaccount.com


created key [9a0bbdfea00c14193e77b1873e25db3263e6e40e] of type [json] as [key.json] for [my-vision-sa@fml-gcp-project-01.iam.gserviceaccount.com]


In [6]:
!set GOOGLE_APPLICATION_CREDENTIALS="D:\Portfolio\my-projects\GCP\key.json"

In [7]:
#!pip3 install -U pip google-cloud-vision

In [8]:
!gcloud projects describe fml-gcp-project-01

createTime: '2021-05-23T20:04:09.910Z'
lifecycleState: ACTIVE
name: fml-gcp-project-01
projectId: fml-gcp-project-01
projectNumber: '318144289060'


In [9]:
from google.cloud import vision

image_uri = 'gs://cloud-samples-data/vision/using_curl/shanghai.jpeg'

client = vision.ImageAnnotatorClient()
image = vision.Image()
image.source.image_uri = image_uri

response = client.label_detection(image=image)

print('Labels (and confidence score):')
print('=' * 30)
for label in response.label_annotations:
    print(label.description, '(%.2f%%)' % (label.score*100.))


Labels (and confidence score):
Wheel (97.90%)
Tire (97.85%)
Bicycle (94.53%)
Photograph (94.24%)
Motor vehicle (91.37%)
Infrastructure (89.83%)
Bicycle wheel (85.65%)
Mode of transport (85.51%)
Asphalt (82.82%)
Umbrella (81.78%)


8. Perform Text Detection
Text detection performs Optical Character Recognition (OCR). It detects and extracts text within an image with support for a broad range of languages. It also features automatic language identification.

In this example, you will perform text detection on an image of an Otter Crossing. Copy the following snippet into your IPython session (or save locally as text_dectect.py):

In [10]:
from __future__ import print_function
from google.cloud import vision

image_uri = 'gs://cloud-vision-codelab/otter_crossing.jpg'

client = vision.ImageAnnotatorClient()
image = vision.Image()
image.source.image_uri = image_uri

response = client.text_detection(image=image)

for text in response.text_annotations:
    print('=' * 30)
    print(text.description)
    vertices = ['(%s,%s)' % (v.x, v.y) for v in text.bounding_poly.vertices]
    print('bounds:', ",".join(vertices))

CAUTION
Otters crossing
for next 6 miles

bounds: (61,243),(251,243),(251,340),(61,340)
CAUTION
bounds: (75,245),(235,243),(235,269),(75,271)
Otters
bounds: (65,296),(140,297),(140,315),(65,314)
crossing
bounds: (151,295),(247,297),(247,318),(151,316)
for
bounds: (61,322),(94,322),(94,340),(61,340)
next
bounds: (106,322),(156,322),(156,340),(106,340)
6
bounds: (167,321),(180,321),(180,339),(167,339)
miles
bounds: (191,321),(251,321),(251,339),(191,339)


9. Perform Landmark Detection
Landmark detection detects popular natural and man-made structures within an image.

In [11]:
from __future__ import print_function
from google.cloud import vision

image_uri = 'gs://cloud-vision-codelab/eiffel_tower.jpg'

client = vision.ImageAnnotatorClient()
image = vision.Image()
image.source.image_uri = image_uri

response = client.landmark_detection(image=image)

for landmark in response.landmark_annotations:
    print('=' * 30)
    print(landmark)

mid: "/g/120xtw6z"
description: "Trocad\303\251ro Gardens"
score: 0.91856456
bounding_poly {
  vertices {
    x: 339
    y: 54
  }
  vertices {
    x: 531
    y: 54
  }
  vertices {
    x: 531
    y: 371
  }
  vertices {
    x: 339
    y: 371
  }
}
locations {
  lat_lng {
    latitude: 48.861596299999995
    longitude: 2.2892823
  }
}

mid: "/m/02j81"
description: "Eiffel Tower"
score: 0.6049596
bounding_poly {
  vertices {
    x: 448
    y: 180
  }
  vertices {
    x: 531
    y: 180
  }
  vertices {
    x: 531
    y: 299
  }
  vertices {
    x: 448
    y: 299
  }
}
locations {
  lat_lng {
    latitude: 48.858461
    longitude: 2.294351
  }
}

mid: "/m/02j81"
description: "Eiffel Tower"
score: 0.5975549
bounding_poly {
  vertices {
    x: 448
    y: 76
  }
  vertices {
    x: 513
    y: 76
  }
  vertices {
    x: 513
    y: 281
  }
  vertices {
    x: 448
    y: 281
  }
}
locations {
  lat_lng {
    latitude: 48.875072
    longitude: 2.312622
  }
}



10. Perform Emotional Face Detection
Facial features detection detects multiple faces within an image along with the associated key facial attributes such as emotional state or wearing headwear.

In this example, you will detect the likelihood of emotional state from four different emotional likelihoods including: joy, anger, sorrow, and surprise.

To perform emotional face detection, copy the following Python code into your IPython session (or save locally as face_dectect.py):

In [12]:
from __future__ import print_function
from google.cloud import vision

uri_base = 'gs://cloud-vision-codelab'
pics = ('face_surprise.jpg', 'face_no_surprise.png')

client = vision.ImageAnnotatorClient()
image = vision.Image()

for pic in pics:
    image.source.image_uri = '%s/%s' % (uri_base, pic)
    response = client.face_detection(image=image)

    print('=' * 30)
    print('File:', pic)
    for face in response.face_annotations:
        likelihood = vision.Likelihood(face.surprise_likelihood)
        vertices = ['(%s,%s)' % (v.x, v.y) for v in face.bounding_poly.vertices]
        print('Face surprised:', likelihood.name)
        print('Face bounds:', ",".join(vertices))

File: face_surprise.jpg
Face surprised: LIKELY
Face bounds: (93,425),(520,425),(520,922),(93,922)
File: face_no_surprise.png
Face surprised: VERY_UNLIKELY
Face bounds: (120,0),(334,0),(334,198),(120,198)


https://github.com/googlecodelabs/cloud-vision-python


In [19]:
#Combine at least 2 of them into another script. For example, 
# add OCR/text recognition to the first script that performs label detection (label_detect.py). 

from google.cloud import vision_v1

#image_uri = 'gs://cloud-samples-data/vision/using_curl/shanghai.jpeg'


input_image_uri="gs://cloud-samples-data/vision/using_curl/shanghai.jpeg"
output_uri="gs://pedok-ml-gcp-vision-api/"

"""Perform async batch image annotation."""
client = vision_v1.ImageAnnotatorClient()

source = {"image_uri": input_image_uri}
image = {"source": source}
features = [
        {"type_": vision_v1.Feature.Type.LABEL_DETECTION},
        {"type_": vision_v1.Feature.Type.TEXT_DETECTION},
    ]

# Each requests element corresponds to a single image.  To annotate more
# images, create a request element for each image and add it to
# the array of requests
requests = [{"image": image, "features": features}]
gcs_destination = {"uri": output_uri}

# The max number of responses to output in each JSON file
batch_size = 2
output_config = {"gcs_destination": gcs_destination,
                     "batch_size": batch_size}

operation = client.async_batch_annotate_images(requests=requests, output_config=output_config)

print("Waiting for operation to complete...")
response = operation.result(90)

# The output is written to GCS with the provided output_uri as prefix
gcs_output_uri = response.output_config.gcs_destination.uri
print("Output written to GCS with prefix: {}".format(gcs_output_uri))
    


Waiting for operation to complete...
Output written to GCS with prefix: gs://pedok-ml-gcp-vision-api/


In [23]:
print(operation.result())

output_config {
  gcs_destination {
    uri: "gs://pedok-ml-gcp-vision-api/"
  }
  batch_size: 2
}



In [26]:
from google.cloud import vision

client = vision.ImageAnnotatorClient()
image_path = 'gs://cloud-samples-data/vision/using_curl/shanghai.jpeg'

source = {"image_uri": image_path}
image = {"source": source}
features = [
        {"type": vision_v1.Feature.Type.LABEL_DETECTION},
        {"type": vision_v1.Feature.Type.TEXT_DETECTION},
    ]

query = [{"image": image, "features": features}]

response = client.annotate_image(query)
print(response.annotations)

#print('Labels (and confidence score):')
#print('=' * 30)
#for label in response.label_annotations:
#    print(label.description, '(%.2f%%)' % (label.score*100.))


TypeError: get() expected a dict or protobuf message, got <class 'list'>.